In [1]:
#automatically reload stuff
%reload_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import re
from Constants import Const
pd.set_option('display.max_columns', None)

In [49]:
def make_symptom_df(starting_index=13, num_attributes = 28):
    #no idea
    df =pd.read_csv(Const.mdasi_folder + 'MDASI_short.csv')

    for i, row in df.iterrows():
        crt_list = row[starting_index:starting_index + num_attributes]    
        s = 0
        for elem in crt_list:
            if pd.isna(elem):
                s += 1


        if s < 14:
            for j in range(len(crt_list)):
                if pd.isna(crt_list[j]):
                    df.iloc[i, starting_index + j] = 0


    for k in range(1,12):
        starting_index = 13+28*k+1


        for i, row in df.iterrows():

            crt_list = row[starting_index:starting_index + num_attributes]    
            s = 0
            for elem in crt_list:
                if pd.isna(elem):
                    s += 1
            if s < 14:
                for j in range(len(crt_list)):
                    if pd.isna(crt_list[j]):
                        prev = df.iloc[i, starting_index + j -28]
                        prev_col_n=df.columns[[1,starting_index + j -28]]
                        prev_ind = starting_index + j -28
                        while pd.isna(prev) == True and prev_ind > 13+28:
                            prev_ind = prev_ind - 28
                            prev = df.iloc[i, prev_ind]
    #                                 prev_col_n=df.columns[[1,prev_ind]]
    #                                 print(prev_ind, prev, prev_col_n)
                        if pd.isna(prev):
                            prev = 0
        #                     df.iloc[i, starting_index + j] = df.iloc[i, starting_index + j -28]
                        df.iloc[i, starting_index + j] = prev
    return df

df = make_symptom_df()
df.head()

,id,age,sex,site_of_tumor,t_numeric,n_nominal,m,ic,rt_dose,rt_fraction,concurrent,nd,RT_duration,baseline_mdasi_pain,baseline_mdasi_fatigue,baseline_mdasi_nausea,baseline_mdasi_sleep,baseline_mdasi_distress,baseline_mdasi_sob,baseline_mdasi_memory,baseline_mdasi_appetite,baseline_mdasi_drowsy,baseline_mdasi_drymouth,baseline_mdasi_sad,baseline_mdasi_vomit,baseline_mdasi_numb,baseline_mdasi_mucus,baseline_mdasi_swallow,baseline_mdasi_choke,baseline_mdasi_voice,baseline_mdasi_skin,baseline_mdasi_constipation,baseline_mdasi_taste,baseline_mdasi_mucositis,baseline_mdasi_teeth,baseline_general_activity,baseline_mdasi_mood,baseline_mdasi_work,baseline_mdasi_relations,baseline_mdasi_walking,baseline_mdasi_enjoy,startRT_mdasi_pain,startRT_mdasi_fatigue,startRT_mdasi_nausea,startRT_mdasi_sleep,startRT_mdasi_distress,startRT_mdasi_sob,startRT_mdasi_memory,startRT_mdasi_appetite,startRT_mdasi_drowsy,startRT_mdasi_drymouth,startRT_mdasi_sad,startRT_mdasi_vomit,startRT_mdasi_numb,startRT_mdasi_mucus,startRT_mdasi_swallow,startRT_mdasi_choke,startRT_mdasi_voice,startRT_mdasi_skin,startRT_mdasi_constipation,startRT_mdasi_taste,startRT_mdasi_mucositis,startRT_mdasi_teeth,startRT_general_activity,startRT_mdasi_mood,startRT_mdasi_work,startRT_mdasi_relations,startRT_mdasi_walking,startRT_mdasi_enjoy,wk_2_mdasi_pain,wk_2_mdasi_fatigue,wk_2_mdasi_nausea,wk_2_mdasi_sleep,wk_2_mdasi_distress,wk_2_mdasi_sob,wk_2_mdasi_memory,wk_2_mdasi_appetite,wk_2_mdasi_drowsy,wk_2_mdasi_drymouth,wk_2_mdasi_sad,wk_2_mdasi_vomit,wk_2_mdasi_numb,wk_2_mdasi_mucus,wk_2_mdasi_swallow,wk_2_mdasi_choke,wk_2_mdasi_voice,wk_2_mdasi_skin,wk_2_mdasi_constipation,wk_2_mdasi_taste,wk_2_mdasi_mucositis,wk_2_mdasi_teeth,wk_2_general_activity,wk_2_mdasi_mood,wk_2_mdasi_work,wk_2_mdasi_relations,wk_2_mdasi_walking,wk_2_mdasi_enjoy,wk_3_mdasi_pain,wk_3_mdasi_fatigue,wk_3_mdasi_nausea,wk_3_mdasi_sleep,wk_3_mdasi_distress,wk_3_mdasi_sob,wk_3_mdasi_memory,wk_3_mdasi_appetite,wk_3_mdasi_drowsy,wk_3_mdasi_drymouth,wk_3_mdasi_sad,wk_3_mdasi_vomit,wk_3_mdasi_numb,wk_3_mdasi_mucus,wk_3_mdasi_swallow,wk_3_mdasi_choke,wk_3_mdasi_voice,wk_3_mdasi_skin,wk_3_mdasi_constipation,wk_3_mdasi_taste,wk_3_mdasi_mucositis,wk_3_mdasi_teeth,wk_3_general_activity,wk_3_mdasi_mood,wk_3_mdasi_work,wk_3_mdasi_relations,wk_3_mdasi_walking,wk_3_mdasi_enjoy,wk_4_mdasi_pain,wk_4_mdasi_fatigue,wk_4_mdasi_nausea,wk_4_mdasi_sleep,wk_4_mdasi_distress,wk_4_mdasi_sob,wk_4_mdasi_memory,wk_4_mdasi_appetite,wk_4_mdasi_drowsy,wk_4_mdasi_drymouth,wk_4_mdasi_sad,wk_4_mdasi_vomit,wk_4_mdasi_numb,wk_4_mdasi_mucus,wk_4_mdasi_swallow,wk_4_mdasi_choke,wk_4_mdasi_voice,wk_4_mdasi_skin,wk_4_mdasi_constipation,wk_4_mdasi_taste,wk_4_mdasi_mucositis,wk_4_mdasi_teeth,wk_4_general_activity,wk_4_mdasi_mood,wk_4_mdasi_work,wk_4_mdasi_relations,wk_4_mdasi_walking,wk_4_mdasi_enjoy,wk_5_mdasi_pain,wk_5_mdasi_fatigue,wk_5_mdasi_nausea,wk_5_mdasi_sleep,wk_5_mdasi_distress,wk_5_mdasi_sob,wk_5_mdasi_memory,wk_5_mdasi_appetite,wk_5_mdasi_drowsy,wk_5_mdasi_drymouth,wk_5_mdasi_sad,wk_5_mdasi_vomit,wk_5_mdasi_numb,wk_5_mdasi_mucus,wk_5_mdasi_swallow,wk_5_mdasi_choke,wk_5_mdasi_voice,wk_5_mdasi_skin,wk_5_mdasi_constipation,wk_5_mdasi_taste,wk_5_mdasi_mucositis,wk_5_mdasi_teeth,wk_5_general_activity,wk_5_mdasi_mood,wk_5_mdasi_work,wk_5_mdasi_relations,wk_5_mdasi_walking,wk_5_mdasi_enjoy,wk_6_mdasi_pain,wk_6_mdasi_fatigue,wk_6_mdasi_nausea,wk_6_mdasi_sleep,wk_6_mdasi_distress,wk_6_mdasi_sob,wk_6_mdasi_memory,wk_6_mdasi_appetite,wk_6_mdasi_drowsy,wk_6_mdasi_drymouth,wk_6_mdasi_sad,wk_6_mdasi_vomit,wk_6_mdasi_numb,wk_6_mdasi_mucus,wk_6_mdasi_swallow,wk_6_mdasi_choke,wk_6_mdasi_voice,wk_6_mdasi_skin,wk_6_mdasi_constipation,wk_6_mdasi_taste,wk_6_mdasi_mucositis,wk_6_mdasi_teeth,wk_6_general_activity,wk_6_mdasi_mood,wk_6_mdasi_work,wk_6_mdasi_relations,wk_6_mdasi_walking,wk_6_mdasi_enjoy,endRT_mdasi_pain,endRT_mdasi_fatigue,endRT_mdasi_nausea,endRT_mdasi_sleep,endRT_mdasi_distress,endRT_mdasi_sob,endRT_mdasi_memory,endRT_mdasi_appetite,endRT_mdasi_drowsy,en

In [ ]:
#Andrews code for formatting the symptom data

In [68]:
def symptom_weektime(string):
    #parses the column for each symptom into the number of weeks after baseline
    s = string.lower().strip().replace('_','')
    if 'baseline' in s:
        return 0
    elif 'startrt' in s:
        return 1
    elif 'endrt' in s:
        return 7
    week_regex = re.match("wk(\d+)",s)
    postweek_regex = re.match('wk(\d+)post',s)
    month_regex = re.match('m(\d+)',s)
    if postweek_regex is not None:
        return 7 + int(postweek_regex.group(1))
    elif week_regex is not None:
        return int(week_regex.group(1))
    elif month_regex is not None:
        return 7 + int(4.35*float(month_regex.group(1)))
    return -1

def get_symptom(string):
    s = string.lower().strip()
    regex = re.match('.*mdasi_([a-z]+)',s)
    if regex is not None:
        return regex.group(1)
    else:
        return None
    
def get_dates(columns):
    alldates = set([])
    for col in columns:
        if 'mdasi' in col:
            week = symptom_weektime(str(col))
            alldates.add(week)
    return sorted(alldates)

def rename_symptom_dict_columns(df):
    #placehold for renaming columns later
    return df
    
def symptom_dict(df,
                 to_keep_clinical = None, 
                 to_keep_demographics = None, 
                 default_value = np.nan,
                symptoms = None):
    df = rename_symptom_dict_columns(df)
    entrys = {}
    if to_keep_clinical is None:
        to_keep_clinical = ['site_of_tumor','t_numeric','rt_dose','rt_fraction','treatment']
    if to_keep_demographics is None:
        to_keep_demographics = ['age','sex']
    if symptoms is None:
        symptoms = Const.symptoms
    dates = get_dates(df.columns)
    for i, row in df.iterrows():
        pentry = row[to_keep_clinical].to_dict()
        pentry['demographics'] = row[to_keep_demographics].to_dict()
        cols = list(row.index)
        p_symptoms = {}
        all_nan = True
        for col in cols:
            if 'mdasi' in col:
                week = symptom_weektime(str(col))
                symp = get_symptom(str(col))
                assert(symp in symptoms)
                entry = p_symptoms.get(symp,{})
                entry[week] = row[col]
                if not np.isnan(row[col]):
                    all_nan = False
                p_symptoms[symp] = entry
        pentry['symptoms'] = {s:[p_symptoms[s].get(d,default_value) for d in dates] for s in p_symptoms.keys()}
        if not all_nan:
            entrys[row['id']] = pentry
        else:
            print(row['id'],'missing all symtpom data')
    ddict = {'dates': dates, 'patients': entrys}
    return ddict
ddict = symptom_dict(df)
ddict

298 missing all symtpom data


{'dates': [0, 1, 2, 3, 4, 5, 6, 7, 13, 33, 59, 85],
 'patients': {1: {'site_of_tumor': 'BOT',
   't_numeric': 1.0,
   'rt_dose': 70.0,
   'rt_fraction': 33.0,
   'treatment': nan,
   'demographics': {'age': '72', 'sex': 2.0},
   'symptoms': {'pain': [1.0,
     nan,
     nan,
     nan,
     nan,
     nan,
     5.0,
     7.0,
     3.0,
     0.0,
     0.0,
     nan],
    'fatigue': [1.0, nan, nan, nan, nan, 1.0, 3.0, 6.0, 7.0, 1.0, nan, nan],
    'nausea': [0.0, nan, nan, nan, nan, 0.0, 2.0, 0.0, 0.0, 0.0, nan, nan],
    'sleep': [1.0, nan, nan, nan, nan, 0.0, 4.0, 7.0, 8.0, 0.0, nan, nan],
    'distress': [1.0, nan, nan, nan, nan, 0.0, 1.0, 1.0, 0.0, 0.0, nan, nan],
    'sob': [0.0, nan, nan, nan, nan, 0.0, 0.0, 0.0, 0.0, 0.0, nan, nan],
    'memory': [0.0, nan, nan, nan, nan, 0.0, 2.0, 0.0, 0.0, 0.0, nan, nan],
    'appetite': [0.0, nan, nan, nan, nan, 0.0, 3.0, 7.0, 4.0, 1.0, nan, nan],
    'drowsy': [0.0, nan, nan, nan, nan, 0.0, 1.0, 5.0, 2.0, 0.0, nan, nan],
    'drymouth': [0.0, na

In [ ]:
#All Carlas Old code compiled into functions

In [ ]:
#check for patients with no symptom ratings

def check_no_symptoms(dframe):
    no_symptoms = []
    for i, row in dframe.iterrows():
        if pd.isna(row['baseline_mdasi_pain']) and \
        pd.isna(row['baseline_mdasi_fatigue']) and \
        pd.isna(row['baseline_mdasi_nausea']) and \
        pd.isna(row['baseline_mdasi_sleep']) and \
        pd.isna(row['startRT_mdasi_pain']) and \
        pd.isna(row['startRT_mdasi_fatigue']) and \
        pd.isna(row['startRT_mdasi_nausea']) and \
        pd.isna(row['startRT_mdasi_sleep']) and \
        pd.isna(row['wk_2_mdasi_pain']) and \
        pd.isna(row['wk_2_mdasi_fatigue']) and \
        pd.isna(row['wk_2_mdasi_nausea']) and \
        pd.isna(row['wk_2_mdasi_sleep']) and \
        pd.isna(row['wk_3_mdasi_pain']) and \
        pd.isna(row['wk_3_mdasi_fatigue']) and \
        pd.isna(row['wk_3_mdasi_nausea']) and \
        pd.isna(row['wk_3_mdasi_sleep']) and \
        pd.isna(row['wk_4_mdasi_pain']) and \
        pd.isna(row['wk_4_mdasi_fatigue']) and \
        pd.isna(row['wk_4_mdasi_nausea']) and \
        pd.isna(row['wk_4_mdasi_sleep']) and \
        pd.isna(row['wk_5_mdasi_pain']) and \
        pd.isna(row['wk_5_mdasi_fatigue']) and \
        pd.isna(row['wk_5_mdasi_nausea']) and \
        pd.isna(row['wk_5_mdasi_sleep']) and \
        pd.isna(row['wk_6_mdasi_pain']) and \
        pd.isna(row['wk_6_mdasi_fatigue']) and \
        pd.isna(row['wk_6_mdasi_nausea']) and \
        pd.isna(row['wk_6_mdasi_sleep']) and \
        pd.isna(row['endRT_mdasi_pain']) and \
        pd.isna(row['endRT_mdasi_fatigue']) and \
        pd.isna(row['endRT_mdasi_nausea']) and \
        pd.isna(row['endRT_mdasi_sleep']) and \
        pd.isna(row['wk6_post_mdasi_pain']) and \
        pd.isna(row['wk6_post_mdasi_fatigue']) and \
        pd.isna(row['wk6_post_mdasi_nausea']) and \
        pd.isna(row['wk6_post_mdasi_sleep']) and \
        pd.isna(row['M6_mdasi_pain']) and \
        pd.isna(row['M6_mdasi_fatigue']) and \
        pd.isna(row['M6_mdasi_nausea']) and \
        pd.isna(row['M6_mdasi_sleep']) and \
        pd.isna(row['M12_mdasi_pain']) and \
        pd.isna(row['M12_mdasi_fatigue']) and \
        pd.isna(row['M12_mdasi_nausea']) and \
        pd.isna(row['M12_mdasi_sleep']) and \
        pd.isna(row['M18_mdasi_pain']) and \
        pd.isna(row['M18_mdasi_fatigue']) and \
        pd.isna(row['M18_mdasi_nausea']) and \
        pd.isna(row['M18_mdasi_sleep']):
            no_symptoms.append(row.id)
    return no_symptoms

def replace_symptom_nan(dframe):
    #replace NaN values for ic and cc with 0 
    d = dframe.copy()
    for i, row in d.iterrows():
        if pd.isna(row['ic']): 
            d.at[i,'ic'] = 0
        if pd.isna(row['concurrent']): 
            d.at[i,'concurrent'] = 0
    return d

def replace_symptom_names(dframe):
    rename_dict = {'concurrent':'cc',
                   'baseline_mdasi_pain':"0_pain", 'baseline_mdasi_fatigue':"0_fatigue", 
                   'baseline_mdasi_nausea':"0_nausea",  'baseline_mdasi_sleep':"0_sleep", 
                   'baseline_mdasi_distress':"0_distress", 'baseline_mdasi_sob':"0_sob", 
                   'baseline_mdasi_memory':"0_memory", 'baseline_mdasi_appetite':"0_appetite", 
                   'baseline_mdasi_drowsy':"0_drowsy",  'baseline_mdasi_drymouth':"0_drymouth", 
                   'baseline_mdasi_sad':"0_sadness", 'baseline_mdasi_vomit':"0_vomit", 
                   'baseline_mdasi_numb':"0_numbness", 'baseline_mdasi_mucus':"0_mucus", 
                   'baseline_mdasi_swallow':"0_swallow",'baseline_mdasi_choke':"0_choking", 
                   'baseline_mdasi_voice':"0_voice",'baseline_mdasi_skin':"0_skin", 
                   'baseline_mdasi_constipation':"0_constipation", 'baseline_mdasi_taste':"0_taste",  
                   'baseline_mdasi_mucositis':"0_mucositis", 'baseline_mdasi_teeth':"0_teeth", 
                   'baseline_general_activity':"0_activity", 'baseline_mdasi_mood':"0_mood", 'baseline_mdasi_work':"0_work", 
                   'baseline_mdasi_relations':"0_relations",'baseline_mdasi_walking':"0_walking",
                   'baseline_mdasi_enjoy':"0_enjoyment",
                   'startRT_mdasi_pain':"1_pain", 'startRT_mdasi_fatigue':"1_fatigue", 'startRT_mdasi_nausea':"1_nausea",
                   'startRT_mdasi_sleep':"1_sleep", 'startRT_mdasi_distress':"1_distress",
                    'startRT_mdasi_sob':"1_sob", 'startRT_mdasi_memory':"1_memory",
                   'wk_2_mdasi_pain':"2_pain", 'wk_2_mdasi_fatigue':"2_fatigue", 
                   'wk_2_mdasi_nausea':"2_nausea",  'wk_2_mdasi_sleep':"2_sleep", 
                   'wk_2_mdasi_distress':"2_distress", 'wk_2_mdasi_sob':"2_sob", 'wk_2_mdasi_memory':"2_memory", 
                   'wk_2_mdasi_appetite':"2_appetite", 'wk_2_mdasi_drowsy':"2_drowsy",  'wk_2_mdasi_drymouth':"2_drymouth", 
                   'wk_2_mdasi_sad':"2_sadness", 'wk_2_mdasi_vomit':"2_vomit", 'wk_2_mdasi_numb':"2_numbness", 'wk_2_mdasi_mucus':"2_mucus", 
                   'wk_2_mdasi_swallow':"2_swallow",'wk_2_mdasi_choke':"2_choking", 'wk_2_mdasi_voice':"2_voice",'wk_2_mdasi_skin':"2_skin", 
                   'wk_2_mdasi_constipation':"2_constipation", 'wk_2_mdasi_taste':"2_taste",  'wk_2_mdasi_mucositis':"2_mucositis",
                   'wk_2_mdasi_teeth':"2_teeth", 'wk_2_general_activity':"2_activity", 'wk_2_mdasi_mood':"2_mood", 'wk_2_mdasi_work':"2_work", 
                   'startRT_mdasi_appetite':"1_appetite", 'startRT_mdasi_drowsy':"1_drowsy",  'startRT_mdasi_drymouth':"1_drymouth",
                   'wk_2_mdasi_relations':"2_relations",'wk_2_mdasi_walking':"2_walking",'wk_2_mdasi_enjoy':"2_enjoyment",
                   'startRT_mdasi_sad':"1_sadness", 'startRT_mdasi_vomit':"1_vomit", 'startRT_mdasi_numb':"1_numbness", 'startRT_mdasi_mucus':"1_mucus",
                   'startRT_mdasi_swallow':"1_swallow",'startRT_mdasi_choke':"1_choking", 'startRT_mdasi_voice':"1_voice",'startRT_mdasi_skin':"1_skin",
                   'startRT_mdasi_constipation':"1_constipation", 'startRT_mdasi_taste':"1_taste",  'startRT_mdasi_mucositis':"1_mucositis",
                   'startRT_mdasi_teeth':"1_teeth", 'startRT_general_activity':"1_activity", 'startRT_mdasi_mood':"1_mood",
                   'startRT_mdasi_work':"1_work", 'startRT_mdasi_relations':"1_relations",'startRT_mdasi_walking':"1_walking",
                   'startRT_mdasi_enjoy':"1_enjoyment",
                  }
    #rename concurrent column with cc
    d = dframe.rename(columns = {'concurrent':"cc"}, inplace=False)
    d = d.rename(columns = {,  ,, , , , , , ,',}, inplace=False)
    d = d.rename(columns = {,}, inplace=False)
    d = d.rename(columns = {'wk_3_mdasi_pain':"3_pain", 'wk_3_mdasi_fatigue':"3_fatigue", 'wk_3_mdasi_nausea':"3_nausea",  'wk_3_mdasi_sleep':"3_sleep", 'wk_3_mdasi_distress':"3_distress", 'wk_3_mdasi_sob':"3_sob", 'wk_3_mdasi_memory':"3_memory", 'wk_3_mdasi_appetite':"3_appetite", 'wk_3_mdasi_drowsy':"3_drowsy",  'wk_3_mdasi_drymouth':"3_drymouth", 'wk_3_mdasi_sad':"3_sadness", 'wk_3_mdasi_vomit':"3_vomit", 'wk_3_mdasi_numb':"3_numbness", 'wk_3_mdasi_mucus':"3_mucus", 'wk_3_mdasi_swallow':"3_swallow",'wk_3_mdasi_choke':"3_choking", 'wk_3_mdasi_voice':"3_voice",'wk_3_mdasi_skin':"3_skin", 'wk_3_mdasi_constipation':"3_constipation", 'wk_3_mdasi_taste':"3_taste",  'wk_3_mdasi_mucositis':"3_mucositis", 'wk_3_mdasi_teeth':"3_teeth", 'wk_3_general_activity':"3_activity", 'wk_3_mdasi_mood':"3_mood", 'wk_3_mdasi_work':"3_work", 'wk_3_mdasi_relations':"3_relations",'wk_3_mdasi_walking':"3_walking",'wk_3_mdasi_enjoy':"3_enjoyment",}, inplace=False)
    d = d.rename(columns = {'wk_4_mdasi_pain':"4_pain", 'wk_4_mdasi_fatigue':"4_fatigue", 'wk_4_mdasi_nausea':"4_nausea",  'wk_4_mdasi_sleep':"4_sleep", 'wk_4_mdasi_distress':"4_distress", 'wk_4_mdasi_sob':"4_sob", 'wk_4_mdasi_memory':"4_memory", 'wk_4_mdasi_appetite':"4_appetite", 'wk_4_mdasi_drowsy':"4_drowsy",  'wk_4_mdasi_drymouth':"4_drymouth", 'wk_4_mdasi_sad':"4_sadness", 'wk_4_mdasi_vomit':"4_vomit", 'wk_4_mdasi_numb':"4_numbness", 'wk_4_mdasi_mucus':"4_mucus", 'wk_4_mdasi_swallow':"4_swallow",'wk_4_mdasi_choke':"4_choking", 'wk_4_mdasi_voice':"4_voice",'wk_4_mdasi_skin':"4_skin", 'wk_4_mdasi_constipation':"4_constipation", 'wk_4_mdasi_taste':"4_taste",  'wk_4_mdasi_mucositis':"4_mucositis", 'wk_4_mdasi_teeth':"4_teeth", 'wk_4_general_activity':"4_activity", 'wk_4_mdasi_mood':"4_mood", 'wk_4_mdasi_work':"4_work", 'wk_4_mdasi_relations':"4_relations",'wk_4_mdasi_walking':"4_walking",'wk_4_mdasi_enjoy':"4_enjoyment",}, inplace=False)
    d = d.rename(columns = {'wk_5_mdasi_pain':"5_pain", 'wk_5_mdasi_fatigue':"5_fatigue", 'wk_5_mdasi_nausea':"5_nausea",  'wk_5_mdasi_sleep':"5_sleep", 'wk_5_mdasi_distress':"5_distress", 'wk_5_mdasi_sob':"5_sob", 'wk_5_mdasi_memory':"5_memory", 'wk_5_mdasi_appetite':"5_appetite", 'wk_5_mdasi_drowsy':"5_drowsy",  'wk_5_mdasi_drymouth':"5_drymouth", 'wk_5_mdasi_sad':"5_sadness", 'wk_5_mdasi_vomit':"5_vomit", 'wk_5_mdasi_numb':"5_numbness", 'wk_5_mdasi_mucus':"5_mucus", 'wk_5_mdasi_swallow':"5_swallow",'wk_5_mdasi_choke':"5_choking", 'wk_5_mdasi_voice':"5_voice",'wk_5_mdasi_skin':"5_skin", 'wk_5_mdasi_constipation':"5_constipation", 'wk_5_mdasi_taste':"5_taste",  'wk_5_mdasi_mucositis':"5_mucositis", 'wk_5_mdasi_teeth':"5_teeth", 'wk_5_general_activity':"5_activity", 'wk_5_mdasi_mood':"5_mood", 'wk_5_mdasi_work':"5_work", 'wk_5_mdasi_relations':"5_relations",'wk_5_mdasi_walking':"5_walking",'wk_5_mdasi_enjoy':"5_enjoyment",}, inplace=False)
    d = d.rename(columns = {'wk_6_mdasi_pain':"6_pain", 'wk_6_mdasi_fatigue':"6_fatigue", 'wk_6_mdasi_nausea':"6_nausea",  'wk_6_mdasi_sleep':"6_sleep", 'wk_6_mdasi_distress':"6_distress", 'wk_6_mdasi_sob':"6_sob", 'wk_6_mdasi_memory':"6_memory", 'wk_6_mdasi_appetite':"6_appetite", 'wk_6_mdasi_drowsy':"6_drowsy",  'wk_6_mdasi_drymouth':"6_drymouth", 'wk_6_mdasi_sad':"6_sadness", 'wk_6_mdasi_vomit':"6_vomit", 'wk_6_mdasi_numb':"6_numbness", 'wk_6_mdasi_mucus':"6_mucus", 'wk_6_mdasi_swallow':"6_swallow",'wk_6_mdasi_choke':"6_choking", 'wk_6_mdasi_voice':"6_voice",'wk_6_mdasi_skin':"6_skin", 'wk_6_mdasi_constipation':"6_constipation", 'wk_6_mdasi_taste':"6_taste",  'wk_6_mdasi_mucositis':"6_mucositis", 'wk_6_mdasi_teeth':"6_teeth", 'wk_6_general_activity':"6_activity", 'wk_6_mdasi_mood':"6_mood", 'wk_6_mdasi_work':"6_work", 'wk_6_mdasi_relations':"6_relations",'wk_6_mdasi_walking':"6_walking",'wk_6_mdasi_enjoy':"6_enjoyment",}, inplace=False)
    d = d.rename(columns = {'endRT_mdasi_pain':"7_pain", 'endRT_mdasi_fatigue':"7_fatigue", 'endRT_mdasi_nausea':"7_nausea",  'endRT_mdasi_sleep':"7_sleep", 'endRT_mdasi_distress':"7_distress", 'endRT_mdasi_sob':"7_sob", 'endRT_mdasi_memory':"7_memory", 'endRT_mdasi_appetite':"7_appetite", 'endRT_mdasi_drowsy':"7_drowsy",  'endRT_mdasi_drymouth':"7_drymouth", 'endRT_mdasi_sad':"7_sadness", 'endRT_mdasi_vomit':"7_vomit", 'endRT_mdasi_numb':"7_numbness", 'endRT_mdasi_mucus':"7_mucus", 'endRT_mdasi_swallow':"7_swallow",'endRT_mdasi_choke':"7_choking", 'endRT_mdasi_voice':"7_voice",'endRT_mdasi_skin':"7_skin", 'endRT_mdasi_constipation':"7_constipation", 'endRT_mdasi_taste':"7_taste",  'endRT_mdasi_mucositis':"7_mucositis", 'endRT_mdasi_teeth':"7_teeth", 'endRT_general_activity':"7_activity", 'endRT_mdasi_mood':"7_mood", 'endRT_mdasi_work':"7_work", 'endRT_mdasi_relations':"7_relations",'endRT_mdasi_walking':"7_walking",'endRT_mdasi_enjoy':"7_enjoyment",}, inplace=False)
    d = d.rename(columns = {'wk6_post_mdasi_pain':"13_pain", 'wk6_post_mdasi_fatigue':"13_fatigue", 'wk6_post_mdasi_nausea':"13_nausea",  'wk6_post_mdasi_sleep':"13_sleep", 'wk6_post_mdasi_distress':"13_distress", 'wk6_post_mdasi_sob':"13_sob", 'wk6_post_mdasi_memory':"13_memory", 'wk6_post_mdasi_appetite':"13_appetite", 'wk6_post_mdasi_drowsy':"13_drowsy",  'wk6_post_mdasi_drymouth':"13_drymouth", 'wk6_post_mdasi_sad':"13_sadness", 'wk6_post_mdasi_vomit':"13_vomit", 'wk6_post_mdasi_numb':"13_numbness", 'wk6_post_mdasi_mucus':"13_mucus", 'wk6_post_mdasi_swallow':"13_swallow",'wk6_post_mdasi_choke':"13_choking", 'wk6_post_mdasi_voice':"13_voice",'wk6_post_mdasi_skin':"13_skin", 'wk6_post_mdasi_constipation':"13_constipation", 'wk6_post_mdasi_taste':"13_taste",  'wk6_post_mdasi_mucositis':"13_mucositis", 'wk6_post_mdasi_teeth':"13_teeth", 'wk6_post_general_activity':"13_activity", 'wk6_post_mdasi_mood':"13_mood", 'wk6_post_mdasi_work':"13_work", 'wk6_post_mdasi_relations':"13_relations",'wk6_post_mdasi_walking':"13_walking",'wk6_post_mdasi_enjoy':"13_enjoyment",}, inplace=False)
    d = d.rename(columns = {'M6_mdasi_pain':"p6_pain", 'M6_mdasi_fatigue':"p6_fatigue", 'M6_mdasi_nausea':"p6_nausea",  'M6_mdasi_sleep':"p6_sleep", 'M6_mdasi_distress':"p6_distress", 'M6_mdasi_sob':"p6_sob", 'M6_mdasi_memory':"p6_memory", 'M6_mdasi_appetite':"p6_appetite", 'M6_mdasi_drowsy':"p6_drowsy",  'M6_mdasi_drymouth':"p6_drymouth", 'M6_mdasi_sad':"p6_sadness", 'M6_mdasi_vomit':"p6_vomit", 'M6_mdasi_numb':"p6_numbness", 'M6_mdasi_mucus':"p6_mucus", 'M6_mdasi_swallow':"p6_swallow",'M6_mdasi_choke':"p6_choking", 'M6_mdasi_voice':"p6_voice",'M6_mdasi_skin':"p6_skin", 'M6_mdasi_constipation':"p6_constipation", 'M6_mdasi_taste':"p6_taste",  'M6_mdasi_mucositis':"p6_mucositis", 'M6_mdasi_teeth':"p6_teeth", 'M6_general_activity':"p6_activity", 'M6_mdasi_mood':"p6_mood", 'M6_mdasi_work':"p6_work", 'M6_mdasi_relations':"p6_relations",'M6_mdasi_walking':"p6_walking",'M6_mdasi_enjoy':"p6_enjoyment",}, inplace=False)
    d = d.rename(columns = {'M12_mdasi_pain':"p12_pain", 'M12_mdasi_fatigue':"p12_fatigue", 'M12_mdasi_nausea':"p12_nausea",  'M12_mdasi_sleep':"p12_sleep", 'M12_mdasi_distress':"p12_distress", 'M12_mdasi_sob':"p12_sob", 'M12_mdasi_memory':"p12_memory", 'M12_mdasi_appetite':"p12_appetite", 'M12_mdasi_drowsy':"p12_drowsy",  'M12_mdasi_drymouth':"p12_drymouth", 'M12_mdasi_sad':"p12_sadness", 'M12_mdasi_vomit':"p12_vomit", 'M12_mdasi_numb':"p12_numbness", 'M12_mdasi_mucus':"p12_mucus", 'M12_mdasi_swallow':"p12_swallow",'M12_mdasi_choke':"p12_choking", 'M12_mdasi_voice':"p12_voice",'M12_mdasi_skin':"p12_skin", 'M12_mdasi_constipation':"p12_constipation", 'M12_mdasi_taste':"p12_taste",  'M12_mdasi_mucositis':"p12_mucositis", 'M12_mdasi_teeth':"p12_teeth", 'M12_general_activity':"p12_activity", 'M12_mdasi_mood':"p12_mood", 'M12_mdasi_work':"p12_work", 'M12_mdasi_relations':"p12_relations",'M12_mdasi_walking':"p12_walking",'M12_mdasi_enjoy':"p12_enjoyment",}, inplace=False)
    d = d.rename(columns = {'M18_mdasi_pain':"p18_pain", 'M18_mdasi_fatigue':"p18_fatigue", 'M18_mdasi_nausea':"p18_nausea",  'M18_mdasi_sleep':"p18_sleep", 'M18_mdasi_distress':"p18_distress", 'M18_mdasi_sob':"p18_sob", 'M18_mdasi_memory':"p18_memory", 'M18_mdasi_appetite':"p18_appetite", 'M18_mdasi_drowsy':"p18_drowsy",  'M18_mdasi_drymouth':"p18_drymouth", 'M18_mdasi_sad':"p18_sadness", 'M18_mdasi_vomit':"p18_vomit", 'M18_mdasi_numb':"p18_numbness", 'M18_mdasi_mucus':"p18_mucus", 'M18_mdasi_swallow':"p18_swallow",'M18_mdasi_choke':"p18_choking", 'M18_mdasi_voice':"p18_voice",'M18_mdasi_skin':"p18_skin", 'M18_mdasi_constipation':"p18_constipation", 'M18_mdasi_taste':"p18_taste",  'M18_mdasi_mucositis':"p18_mucositis", 'M18_mdasi_teeth':"p18_teeth", 'M18_general_activity':"p18_activity", 'M18_mdasi_mood':"p18_mood", 'M18_mdasi_work':"p18_work", 'M18_mdasi_relations':"p18_relations",'M18_mdasi_walking':"p18_walking",'M18_mdasi_enjoy':"p18_enjoyment",}, inplace=False)
    return d

def clean_symptom_df(dframe):
    no_symptoms = set(check_no_symptoms(dframe))
    d = dframe[~dframe.id.isin(no_symptoms)]
    d = replace_symptom_nan(d)
    
    #rename concurrent column with cc
    d = replace_symptom_names(d)
    
    #this part is missing <check_one_timepoint>
    #replace baseline missing values with 0 for patients who have at least 1 rating dif than 0 during baseline
#     for i, row in d.iterrows():
#         rez = check_one_timepoint(row[13:41])
#         if rez == True:
#             d.iloc[i-1,13:41] = d.iloc[i-1,13:41].fillna(0)
    return d

df = clean_symptom_df(df)
df

In [ ]:
def symptom_df_to_dict(dframe):
    
    patients = {}
    for i, row in dframe.iterrows():
        for el in row:

            patients[row["id"]] = {}

            patients[row['id']]["demographics"] = {"gender":row["sex"], "age":row["age"]}

            treatment = ""
            if not np.isnan(row['ic']) and pd.to_numeric(row['ic']) == 1:
                treatment += "ic"
            if not np.isnan(row['cc']) and pd.to_numeric(row['cc']) == 1:
                treatment +="cc"
            patients[row['id']]['clinical'] ={"site_of_tumor":row["site_of_tumor"], "t":row['t_numeric'], "n":row["n_nominal"], 'treatment':treatment } 

            patients[row['id']]['symptoms'] = {}
            first_symp_index = 13
            for s_index, s in enumerate(symptoms):
                patients[row['id']]['symptoms'][s] =[]
                for timepoint in range(12):
                    patients[row['id']]['symptoms'][s].append(row[first_symp_index + 28* timepoint + s_index])
    return patients

sdict = symptom_df_to_dict(df)
sdict

In [ ]:
with open(Const.mdasi_folder + "patients.json", "wb") as f:
    f.write(json.dumps(sdict, indent = 4).encode("utf-8"))